### 決定木　パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import  mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree.export import export_text
from sklearn import preprocessing

import graphviz

C:\Users\owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
pd.__version__

'1.0.3'

In [4]:
train_pkl.shape

(55323, 12)

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

### 検証するパラメータの設定

In [7]:
grid_param = {
    'criterion': ['mse'],
    'splitter': ['best', 'random'],
    'max_depth': [i for i in range(1, 11, 2)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [ ]:
grid_param = {
    'criterion': ['mse'],
    'splitter': ['best', 'random'],
    'max_depth': [i for i in range(5, 6)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [ ]:
def rmse(y, y_pred):
    ret = np.sqrt(mean_squared_error(y, y_pred))
    print("RMSE=", ret)
    return ret

### グリッドサーチ

In [8]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(**grid_param), param_grid=grid_param, scoring=make_scorer(mean_squared_error, greater_is_better=False), cv=5, return_train_score=False)

In [9]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion=['mse'],
                                             max_depth=[1, 3, 5, 7, 9],
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=[1, 3, 5, 7, 9],
                                             min_samples_split=[2, 4, 6, 8, 10],
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=[42],
                                             splitter=['best', 'random']),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['mse'], 'max_dept

### 一番良かったパラメータで学習

In [10]:
gs.best_score_

-17298.477617999713

In [11]:
np.sqrt(abs(gs.best_score_))

131.52367702432787

In [12]:
gs.best_params_

{'criterion': 'mse',
 'max_depth': 9,
 'min_samples_leaf': 9,
 'min_samples_split': 2,
 'random_state': 42,
 'splitter': 'best'}

In [13]:
# gs.cv_results_['mean_test_score']

In [14]:
DT = DecisionTreeRegressor(**gs.best_params_)

In [15]:
DT = DT.fit(train_X, train_y)

### ツリーの可視化

In [16]:
# tree.plot_tree(DT.fit(train_X, train_y))

In [17]:
# export_text(DT, feature_names=train.iloc[:, 1:].columns)

In [18]:
# train.Survived.name

In [19]:
# len(train.iloc[:, 1:].columns), train.iloc[:, 1:].columns

In [20]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(DT, out_file=None,
#                                feature_names=train.iloc[:, 1:-1].columns,
#                                class_names=train.y.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

In [21]:
# graph = graphviz.Source(dot_data)
# graph

In [22]:
# graph.write('.\DT.png')

In [23]:
type(train_X)

numpy.ndarray

In [24]:
DT.feature_importances_

array([0.11866497, 0.22739946, 0.47818374, 0.02425542, 0.06305541,
       0.00396233, 0.00127072, 0.00161243, 0.00071202, 0.0808835 ])

In [25]:
# 特徴量の重要度が高い順に表示
print("特徴量の重要度が高い順：")
# sorted：reverse=True 降順
print(sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]].columns),
    reverse=True))

特徴量の重要度が高い順：
[(0.478, 'bedrooms'), (0.227, 'bathrooms'), (0.119, 'accommodates'), (0.081, 'city_flag'), (0.063, 'host_response_rate'), (0.024, 'beds'), (0.004, 'number_of_reviews'), (0.002, 'property_type_num'), (0.001, 'cancel_policy_flag'), (0.001, 'bed_type_flag')]


In [27]:
sorted(
    zip(map(lambda x: round(x, 3), DT.feature_importances_), train.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]].columns),
    reverse=True)

[(0.478, 'bedrooms'),
 (0.227, 'bathrooms'),
 (0.119, 'accommodates'),
 (0.081, 'city_flag'),
 (0.063, 'host_response_rate'),
 (0.024, 'beds'),
 (0.004, 'number_of_reviews'),
 (0.002, 'property_type_num'),
 (0.001, 'cancel_policy_flag'),
 (0.001, 'bed_type_flag')]

### テストデータで予測

In [28]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [29]:
test_x.shape, test_y.shape

((11065, 10), (11065,))

In [30]:
pred_y = DT.predict(test_x)

In [31]:
np.sqrt(mean_squared_error(test_y, pred_y))

132.18771996404405

### 検証データで予測

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [ ]:
valid.shape

In [ ]:
# ID の保存
valid_pass = valid.id.values

In [ ]:
valid_X = valid.iloc[:, 1:].values

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
pred_valid_y = DT.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [ ]:
result_df.to_csv("./tree_9.csv", header=False)